- 사용할 라이브러리 목록

In [1]:
#import할 패키지 목록
import re
import pandas as pd
from tqdm import tqdm
from konlpy.tag import Okt
#from pykospacing import Spacing
from collections import Counter

import numpy as np

import konlpy

In [2]:
import re; import os; import sys; import json
from pykospacing import Spacing
from konlpy.tag import Kkma
from konlpy.tag import Okt
import soynlp
import numpy as np; import pandas as pd


kkma = Kkma(); okt = Okt()

In [3]:
df_top1 = pd.read_table('[ADV][무탠다드Top1]테이퍼드 히든 밴딩 크롭 슬랙스.csv', sep=',')
df_top2 = pd.read_table('[ADV][무탠다드Top2]히든밴딩슬랙스.csv', encoding='cp949',sep=',')

df_top1 = df_top1.iloc[:,1:13]
df_top2 = df_top2.iloc[:,1:13]

- 색깔만 다르고, 동일한 사용자가 동일한 사이즈의 동일 제품을 구매한 경우의 중복을 제거

In [4]:
df_top1['item'] = '테이퍼드 히든 밴딩 크롭 슬랙스'
df_top2['item'] = '히든 밴딩 슬랙스'

df_top1.drop_duplicates(inplace=True)
df_top2.drop_duplicates(inplace=True)

print(df_top1.shape)
print(df_top2.shape)

(884, 12)
(822, 12)


- Top1인 테이퍼드 히든 밴딩 크롭 슬랙스 4943개에서 884개됨................ㅠㅅㅠ
- Top2인 히든 밴딩 슬랙스 1001개에서 822개됨.....

In [5]:
df = pd.concat([df_top1, df_top2])
df.shape

(1706, 12)

- **리뷰 분석에 사용할 컬럼만 추출한 데이터프레임 생성**

In [6]:
df_rv = df.iloc[:,[0,1,2,3,4,5,7,8]]
df_rv.reset_index(drop=True, inplace=True)
df_rv.head()

,user,gender,height,weight,item,size,content,size_eval
0,LV 3 뉴비_f68324b5,남성,174cm,73kg,테이퍼드 히든 밴딩 크롭 슬랙스,31,원래 핏하게 입는데요 31사이즈 딱 좋습니다 예뻐요!,보통이에요
1,LV 3 sup1234,남성,168cm,67kg,테이퍼드 히든 밴딩 크롭 슬랙스,32,기본 테이퍼드 슬랙스로 적합합니다. 기본 바지 없으신 분들께 추천드려요!,보통이에요
2,LV 5 취월,남성,178cm,87kg,테이퍼드 히든 밴딩 크롭 슬랙스,38,허리가 사이즈 너무 크게 나오긴하는데.. 가성비는 좋습니다.,커요
3,LV 3 뉴비_1c43eb209847,남성,174cm,84kg,테이퍼드 히든 밴딩 크롭 슬랙스,34,아주 무난한 정장바지입니다. 핏도좋고 허리 맞추기도 좋음,보통이에요
4,LV 6 데스티닝,남성,164cm,58kg,테이퍼드 히든 밴딩 크롭 슬랙스,29,항상 재구매하는 아이템입니다 핏 좋아서 만족합니다,보통이에요


- 한글표현만 남기기

In [30]:
#한글표현만 남기기 : 사용자 정의 함수 지정
def extract_word(text):
    hangul = re.compile('[^가-힣]') 
    result = hangul.sub(' ', text) 
    return result

print(df_rv.iloc[0,6])
print(extract_word(df_rv.iloc[0,6]))

원래 핏하게 입는데요 31사이즈 딱 좋습니다 예뻐요!
원래 핏하게 입는데요   사이즈 딱 좋습니다 예뻐요 


In [33]:
df_rv['content'] = df_rv['content'].apply(lambda x : extract_word(x))
df_rv.head()

<ipython-input-33-1fba116224b9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rv['content'] = df_rv['content'].apply(lambda x : extract_word(x))


,user,gender,height,weight,item,size,content,size_eval
0,LV 3 뉴비_f68324b5,남성,174cm,73kg,테이퍼드 히든 밴딩 크롭 슬랙스,31,원래 핏하게 입는데요 사이즈 딱 좋습니다 예뻐요,보통이에요
1,LV 3 sup1234,남성,168cm,67kg,테이퍼드 히든 밴딩 크롭 슬랙스,32,기본 테이퍼드 슬랙스로 적합합니다 기본 바지 없으신 분들께 추천드려요,보통이에요
2,LV 5 취월,남성,178cm,87kg,테이퍼드 히든 밴딩 크롭 슬랙스,38,허리가 사이즈 너무 크게 나오긴하는데 가성비는 좋습니다,커요
3,LV 3 뉴비_1c43eb209847,남성,174cm,84kg,테이퍼드 히든 밴딩 크롭 슬랙스,34,아주 무난한 정장바지입니다 핏도좋고 허리 맞추기도 좋음,보통이에요
4,LV 6 데스티닝,남성,164cm,58kg,테이퍼드 히든 밴딩 크롭 슬랙스,29,항상 재구매하는 아이템입니다 핏 좋아서 만족합니다,보통이에요


- <span style='background-color: #fff5b1'>띄어쓰기 수정 : pykospacing 안깔리는데... 모가문제임.... 이후에 적용</span>

- 이때 konlpy 버전에 따라 Twitter > Okt 로 변경됨
- 한국어 형태소 분석기에는 kkma/ komoran/ okt/ hannanum 존재

In [34]:
#한국어 형태소 분석기
from konlpy.tag import Kkma, Komoran, Okt, Hannanum #Mecab은 윈도우에서 작동 불가능
okt = Okt()
kkma = Kkma()
komoran = Komoran()
hannanum = Hannanum()

In [38]:
#크롤링 된 것에서 특정 리뷰만 가져와보자
text = df_rv.iloc[0,6]+df_rv.iloc[1,6]+df_rv.iloc[2,6]

In [40]:
# .morphs()함수: 텍스트를 형태소 단위로 나누어줌
print("[Kkma morphs 함수]")
print(kkma.morphs(text))
print("[Okt 함수]")
print(okt.morphs(text))
print("[Komoran 함수]")
print(komoran.morphs(text))
print("[Hannanum 함수]")
print(hannanum.morphs(text))

[Kkma morphs 함수]
['원래', '핏', '하', '게', '입', '는데요', '사이즈', '딱', '좋', '습니다', '예쁘', '어요', '기본', '테', '이퍼드', '슬랙스', '로', '적합', '하', 'ㅂ니다', '기본', '바지', '없', '으시', 'ㄴ', '분', '들', '께', '추천', '드', '려요', '허리', '가', '사이즈', '너무', '크', '게', '나오', '기', '는', '하', '는데', '가', '아', '성비', '는', '좋', '습니다']
[Okt 함수]
['원래', '핏하', '게', '입는데요', '사이즈', '딱', '좋습니다', '예뻐요', '기본', '테이', '퍼드', '슬랙스', '로', '적합합니다', '기본', '바지', '없으신', '분', '들', '께', '추천', '드려요', '허리', '가', '사이즈', '너무', '크게', '나오긴하는데', '가성', '비는', '좋습니다']
[Komoran 함수]
['원래', '핏하게', '입', '는데요', '사이즈', '딱', '좋', '습니다', '예쁘', '어요', '기본', '테', '이퍼', '드', '슬랙스로', '적합', '하', 'ㅂ니다', '기본', '바지', '없', '으시', 'ㄴ', '분', '들', '께', '추천', '드리', '어요', '허리', '가', '사이즈', '너무', '크', '게', '나오', '기', 'ㄴ', '하', '는데', '가', '성비', '는', '좋', '습니다']
[Hannanum 함수]
['원래', '핏하', '이', '게', '입', '는데', '요', '사이즈', '딱', '좋', '습니다', '예쁘', '어', '요', '기본', '테이퍼드', '슬랙스', '로', '적합', '하', 'ㅂ니다', '기본', '바', '이', '지', '없', '으시ㄴ', '불', 'ㄴ', '들', '께', '추천드', '이', '리어', '요', '허리', '가', '사이즈',

- 간단하게 3가지 리뷰만 돌려본건데도 Okt 분석기가 가장 좋았음
- 이후부터는 Okt 분석기만 사용
- 일단 전체 리뷰에 대해서 의미있는 것만 살펴보아야하므로, 하나로 모음

In [41]:
okt = Okt()
words = " ".join(df_rv['content'].tolist())
words = okt.morphs(words, stem=True)

In [43]:
#모든 리뷰에 대한 형태소 분석 및 어간 추출
print(words)

['원래', '핏하', '게', '입다', '사이즈', '딱', '좋다', '예쁘다', '기본', '테이', '퍼드', '슬랙스', '로', '적합하다', '기본', '바지', '없다', '분', '들', '께', '추천', '드리다', '허리', '가', '사이즈', '너무', '크게', '나오다', '가성', '비다', '좋다', '아주', '무난', '한', '정장', '바지', '이다', '핏도좋', '고', '허리', '맞추다', '좋다', '항상', '재다', '매하', '는', '아이템', '이다', '핏', '좋다', '만족하다', '면', '이정', '도', '핏', '나오다', '기', '본', '템', '은', '역시', '무탠다드', '인', '것', '같다', '재질', '좋다', '가성', '비', '좋다', '스타일', '내기', '좋다', '감사하다', '슬랙스', '가', '좋다', '들어서다', '구입', '하다', '듣다', '맘', '에', '들다', '바지', '이다', '물다', '스탠다드', '슬랙스', '는', '정말', '가성', '비', '최고', '이다', '테이', '퍼드', '핏', '으로', '딱', '제', '가', '원하다', '핏', '이', '나오다', '무난', '하다', '입다', '슬렉스러', '좋다', '것', '같다', '역시', '슬랙스', '장인', '물다', '스탠다드', '모든', '게', '완벽하다', '색감', '이랑', '사이즈', '랑', '딱', '맞다', '같다', '요', '이러나저러나', '괜츈', '지인', '에게', '선물', '하다', '아주', '맘', '에', '들다', '하다', '체형', '에서', '키', '는', '에', '배다', '조금', '나왓는데', '전체', '적', '으로', '날씬하다', '편', '에요', '길이', '감다', '딱', '떨어지다', '이쁘다', '무난', '하다', '손', '이', '자주', '가다', '바지', '이다'

- 한글 리뷰 자체에서 조사와 어미는 잘 구분되고 있음
- 하지만 가성비, 기본템과 같은 단어는 처리에 어려움이 존재함 (가성+비, 기+본+템)

In [54]:
words = " ".join(df_rv['content'].tolist())

In [55]:
from soynlp.tokenizer import LTokenizer
lto = LTokenizer()
print(lto.tokenize(words))

['원래', '핏하게', '입는데요', '사이즈', '딱', '좋습니다', '예뻐요', '기본', '테이퍼드', '슬랙스로', '적합합니다', '기본', '바지', '없으신', '분들께', '추천드려요', '허리가', '사이즈', '너무', '크게', '나오긴하는데', '가성비는', '좋습니다', '아주', '무난한', '정장바지입니다', '핏도좋고', '허리', '맞추기도', '좋음', '항상', '재구매하는', '아이템입니다', '핏', '좋아서', '만족합니다', '면', '이정도', '핏', '나옵니다', '기본템은', '역시', '무탠다드인', '것', '같습니다', '재질좋고', '가성비', '좋고', '스타일', '내기', '좋습니다', '감사합니다', '슬랙스가', '좋다고', '들어서', '구입했는데', '듣던대로', '맘에', '드는', '바지입니다', '무신사', '스탠다드', '슬랙스는', '정말', '가성비', '최고입니다', '테이퍼드', '핏으로', '딱', '제가', '원하는', '핏이', '나왔고', '무난하게입는', '슬렉스러', '좋은', '것', '같아요', '역시', '슬랙스', '장인', '무신사', '스탠다드', '모든게', '완벽하다', '색감이랑', '사이즈랑', '딱맞는거같음요이러나저러나', '괜츈', '지인에게', '선물했는데', '아주', '맘에든다고하네용체형에서', '키는', '에', '배는', '조금', '나왓는데', '전체적으로', '날씬한편에요길이감도', '딱', '떨어지고', '이쁩니다', '무난해서', '손이', '자주가는', '바지입니다', '너무', '좋아요', '아버지', '사드럈는데', '가성비', '좋게', '잘', '입고다니셔서', '제', '기분이', '다', '젛네요', '만족합니다', '싸게잘삿습니다', '베이지', '슬랙스이뻐요', '추천합니다', '만족합니다', '싸게잘삿습니다', '이뻐요', '추천합니다', '막입기에도', '편하고', '재질도', '아주아주', '편합니다', '

In [56]:
from soynlp.word import WordExtractor

In [61]:
word_ext = WordExtractor()
word_ext.train(df_rv['content'].values)
scores = word_ext.word_scores()

training was done. used memory 0.108 Gbry 0.102 Gb
all cohesion probabilities was computed. # words = 1684
all branching entropies was computed # words = 3811
all accessor variety was computed # words = 3811


In [62]:
import math
score_dict = {key: scores[key].cohesion_forward*math.exp(scores[key].right_branching_entropy)
             for key in scores}

In [64]:
lto = LTokenizer(scores=score_dict)

In [69]:
print(df_rv.iloc[2,6])
print(okt.morphs(df_rv.iloc[2,6]))
print(lto.tokenize(df_rv.iloc[2,6]))

허리가 사이즈 너무 크게 나오긴하는데   가성비는 좋습니다 
['허리', '가', '사이즈', '너무', '크게', '나오긴하는데', '가성', '비는', '좋습니다']
['허리', '가', '사이즈', '너무', '크게', '나오', '긴하는데', '가성비', '는', '좋습니다']


- 의미있는 단어임에도 기존에 잘 잡아주지 못했던 '가성비'를 잘 잡아주고 있음
- 감정분석에 대한 객관적 평가를 위해서는 target 값이 존재해야함
- 사용자가 매긴 평점을 target 값으로 추출해 사용 -> 어떤 값을 기준으로 긍/부정으로 묶을건지

In [74]:
print(df_rv.iloc[3,6])
print(okt.morphs(df_rv.iloc[3,6]))
print(lto.tokenize(df_rv.iloc[3,6]))

아주 무난한 정장바지입니다  핏도좋고 허리 맞추기도 좋음
['아주', '무난', '한', '정장', '바지', '입니다', '핏도좋', '고', '허리', '맞추기도', '좋음']
['아주', '무난', '한', '정장', '바지입니다', '핏도', '좋고', '허리', '맞추기도', '좋음']


In [75]:
print(df_rv.iloc[4,6])
print(okt.morphs(df_rv.iloc[4,6]))
print(lto.tokenize(df_rv.iloc[4,6]))

항상 재구매하는 아이템입니다 핏 좋아서 만족합니다
['항상', '재구', '매하', '는', '아이템', '입니다', '핏', '좋아서', '만족합니다']
['항상', '재구매', '하는', '아이템', '입니다', '핏', '좋아서', '만족합니다']


### 감성분석

In [14]:
#파일을 보기 좋게, 딕셔너리 형태로 변환
rv = []
i=0
for i in range(0, len(df)):
    temp_dict = {}
    temp_dict['_id'] = df.index[i]
    temp_dict['review'] = df.iloc[i,7]
    if temp_dict['review'] == []:
        continue
    rv.append(temp_dict)

In [16]:
#전처리 함수 생성
def preprocessing(review):
    total_review = ''
    #인풋리뷰
    for idx in range(len(review)):
        r = review[idx]
        #하나의 리뷰에서 문장 단위로 자르기
        for sentence in kkma.sentences(r):
            sentence = re.sub('([a-zA-Z])','',sentence)
            sentence = re.sub('[ㄱ-ㅎㅏ-ㅣ]+','',sentence)
            sentence = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]','',sentence)
            if len(sentence) == 0:
                continue
            if len(sentence) < 198:
                sentence = spacing(sentence)  #잘못된 띄어쓰기 처리
            sentence += '. '
            total_review += sentence
    return total_review

In [17]:
#감성분석을 위한 감성사전 만들기
clothes_good_feature = {'사이즈':['최고','감동','좋','괜찮','최고','짱','훌륭','추천',
                               '최상','대박','훈훈','특별','개이득','최고','만족','세련','맞','괜춘','합리적'],
                       '옷':['최고','감동','좋','괜찮','최고','짱','훌륭','추천',
                               '최상','대박','훈훈','특별','개이득','최고','만족','세련','맞','괜춘','합리적'],
                       '바지':['최고','감동','좋','괜찮','최고','짱','훌륭','추천',
                               '최상','대박','훈훈','특별','개이득','최고','만족','세련','맞','괜춘','합리적'],
                       '길이':['최고','감동','좋','괜찮','최고','짱','훌륭','추천',
                               '최상','대박','훈훈','특별','개이득','최고','만족','세련','맞','괜춘','합리적']}
clothes_bad_feature = {'사이즈':['별로','최악','짜증','낌','작','헐렁','나쁘다','흐르다','크다','작다','흘러내리다',
                              '이상하다','이상'],
                      '옷':['별로','최악','짜증','낌','작','헐렁','나쁘다','흐르다','크다','작다','흘러내리다',
                              '이상하다','이상'],
                      '길이':['별로','최악','짜증','낌','작','헐렁','나쁘다','흐르다','크다','작다','흘러내리다',
                              '이상하다','이상'],
                      '바지':['별로','최악','짜증','낌','작','헐렁','나쁘다','흐르다','크다','작다','흘러내리다',
                              '이상하다','이상']}

In [18]:
#특징 키워드를 추출하는 함수
def get_feature_keywords(feature_keywords, review):
    feature_temp = []
    for keyword in feature_keywords:
        if re.findall(keyword, review):
            sub_list = ['게','고','음','며','데','만','도','면']
            
            for sub in sub_list:
                if sub+' ' in review:
                    review = re.sub(sub+' ', sub+',', review)
                
            a = re.findall(keyword +'+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+',review) # K한 한 한글
            b = re.findall(keyword + '+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+',review) # K 한 한글 
            c = re.findall('[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+' + keyword +'[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+',review) # 한 K한글 예쁜 분위기가
                
            for ngram in a:
                t = ()
                feature_temp.append(t + (ngram,keyword))
            for ngram in b:
                t = ()
                feature_temp.append(t + (ngram,keyword))
            for ngram in c:
                t = ()
                feature_temp.append(t + (ngram,keyword))
                
    return feature_temp

In [19]:
def get_feature_emotions(feature_good_dict,feature_bad_dict, feature_temp):
    good_feature_emotion_list = []
    bad_feature_emotion_list = []
    
    for ngrams in feature_temp:
        keyword = ngrams[1]
        ngram = ngrams[0]
        is_bad_feature = None
        
        good_emotion_list = feature_good_dict[keyword]
        bad_emotion_list = feature_bad_dict[keyword]
        for emotion in good_emotion_list:
            if re.findall(emotion, ngram):
                is_bad_feature = False  
        for emotion in bad_emotion_list:
            if re.findall(emotion, ngram):
                is_bad_feature = True    
        for negative in negative_word_emotion:
            if re.findall(negative, ngram):
                if is_bad_feature == True:
                    is_bad_feature = False
                    break
                elif is_bad_feature == False:
                    is_bad_feature = True
                    break
                else:
                    is_bad_feature = True
                    break   
        if is_bad_feature:
            bad_feature_emotion_list.append(ngram)
        elif is_bad_feature == False:
            good_feature_emotion_list.append(ngram)
        else:
            pass
    return good_feature_emotion_list, bad_feature_emotion_list

In [ ]:
get_feature_emotions(clothes_good_feature, clothes_bad_feature, )

In [ ]:
def get_taste_emotion(taste_good_emotions, taste_bad_emotions):
    bad_taste_emotion_list = []
    good_taste_emotion_list = []
    for ngrams in taste_good_emotions:
        ngram = ngrams[0]
        is_bad_taste = False
        for negative in negative_word_emotion:
            if re.findall(negative,ngram):
                is_bad_taste = True
        if is_bad_taste:
            bad_taste_emotion_list.append(ngram)
        else:
            good_taste_emotion_list.append(ngram)
    
    for ngrams in taste_bad_emotions:
        ngram = ngrams[0]
        is_bad_taste = True
        for negative in negative_word_emotion:
            if re.findall(negative,ngram):
                is_bad_taste = False
        if is_bad_taste:
            bad_taste_emotion_list.append(ngram)
        else:
            good_taste_emotion_list.append(ngram)
            
    return good_taste_emotion_list, bad_taste_emotion_list